In [93]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [94]:
product_data = pd.read_csv('../data/market_integrates_1.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         5.533338        20.855216  2.404884   
1             1         2         5.376039        20.650171  2.595498   
2             1         3         5.713958        20.379129  2.627864   
3             1         4         5.033581        20.989227  2.529403   
4             1         5         5.591846        20.380606  2.523367   
..          ...       ...              ...              ...       ...   
995         100         6         5.830374        20.702445  2.465779   
996         100         7         5.946392        20.314758  2.419043   
997         100         8         5.543462        20.189176  2.433673   
998         100         9         5.201490        20.021045  2.729257   
999         100        10         5.888071        20.440777  2.551297   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         1.779635  0.015173  0.009487  1.351336

In [95]:
# # Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
# product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
#                                                     #   'capital':'supply_instruments1',
#                                                         # 'marginal_cost': 'supply_instrument2'
#                                                         })



In [96]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [97]:
demand_instruments = pyblp.build_blp_instruments(X1_formulation, product_data)
df = product_data.join(pd.DataFrame(demand_instruments))

product_data_complete = df.rename(columns={
                                        # 0: 'demand_instrument0',
                                           'labor': 'supply_instrument0', 
                                           'marginal_cost': 'supply_instrument1',
                                        #    1: 'demand_instrument1', 
                                        #    2: 'demand_instrument2', 
                                        #    3: 'demand_instrument3',
                                        #    4: 'demand_instrument4',
                                        #     5: 'demand_instrument5',
                                        #     6: 'demand_instrument6',
                                        #     7:'demand_instrument7'
                                                })



In [98]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [99]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration, add_exogenous=True)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [100]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [101]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           451         1476         0     +5.411187E-17                 +1.662602E-08  +1.000000E+00

Optimization completed after 00:00:00.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:01.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------